In [ ]:
#!/usr/bin/env python3
# -*- coding: UTF-8 -*-

import os
import sys
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER

TKIPDATA="./"

def read_file(f):
    lines = f.read().splitlines()
    return lines

def pos_analysis(lst, ws, pos, ner):
    word_sentence_list = ws(lst, sentence_segmentation=True)
    pos_sentence_list = pos(word_sentence_list)
    
    return(word_sentence_list, pos_sentence_list)

def print_pos(sentence_list, word_sentence_list,  pos_sentence_list, ofp):
    for i, sentence in enumerate(sentence_list):
        #print(sentence, word_sentence_list[i],  pos_sentence_list[i])
        print_word_pos_sentence(sentence_list, word_sentence_list[i],  pos_sentence_list[i], ofp)

# Show results
def print_word_pos_sentence(sentence_list, word_sentence_list, pos_sentence_list, ofp):
    assert len(word_sentence_list) == len(pos_sentence_list)
    for word, pos in zip(word_sentence_list, pos_sentence_list):
        print(f"{word}({pos})", end="\u3000")
        ofp.write(f"{word}({pos}) ")
    print()
    ofp.write("\n")

def tkip_data():
    if not os.path.isdir(TKIPDATA):
        print("Downloading CKIP data...")
        # Download data
        os.makedirs(TKIPDATA)
        data_utils.download_data(TKIPDATA)

    print("Loading CKIP data...")
    # Load model
    ws = WS(TKIPDATA+"data")
    pos = POS(TKIPDATA+"data")
    ner = NER(TKIPDATA+"data")
    
    return (ws, pos, ner)

(ws, pos, ner) = tkip_data()

while(True):
    txtfile=input("Input text file: ")
    posfile=os.path.splitext(txtfile)[0]+'.pos'
    print("Input: ", txtfile, "Output: ", posfile)
    print()
    
    fp = open(txtfile, encoding="utf8")
    ofp= open(posfile, "w", encoding="utf8")

    lines = read_file(fp)
    for l in lines:
        print(l)

    print("-----------------------------")
    (wl, pl) = pos_analysis(lines, ws, pos, ner)
    print_pos(lines, wl, pl, ofp)

    print("-----------DONE--------------")
    fp.close()
    ofp.close()

# Release model
del ws
del pos
del ner

